# Simple NLP for Korean
간단한 Tokenizing, Pos-tagging을 해보고 실제 예제로 konlpy의 예제 corpus로 DocumentTermMatrix를 만들어봅니다. 이 때, DocumentTermMatrix의 Term으로는 초록에 등장한 명사(NN), 고유명사(NNP) 중 많이 출현한 100개의 단어로 Term을 만들고 그 것을 토대로 DocumentTermMatrix를 만들어봅니다.  
  
* _nltk와 sklearn의 sub-module인 feature-extraction.text를 활용합니다._
* _pos-tagger로 konlpy에서 kkma 형태소 분석기를 활용합니다._
* 실제로 활용하셔도 좋으나 조금 더 코드를 다듬어서 사용하시는 것을 추천드립니다.
* nltk : http://www.nltk.org/book/
* konlpy : http://konlpy-ko.readthedocs.io/ko/v0.4.3/  

만든이 : 김보섭

## Pos tagging for Korea

In [1]:
from konlpy.tag import Kkma # 꼬꼬마 형태소분석기
import re
kkma = Kkma()

### Tokenizing & Pos-tagging

In [2]:
# string에 바로 pos tagger 적용
s1 = '비정형데이터분석 수업에서는 정말 많은 것을 배웁니다.'
print(type(s1))
kkma.pos(s1)

<class 'str'>


[('비정형', 'NNG'),
 ('데이터', 'NNG'),
 ('분석', 'NNG'),
 ('수업', 'NNG'),
 ('에서', 'JKM'),
 ('는', 'JX'),
 ('정말', 'MAG'),
 ('많', 'VA'),
 ('은', 'ETD'),
 ('것', 'NNB'),
 ('을', 'JKO'),
 ('배우', 'VV'),
 ('ㅂ니다', 'EFN'),
 ('.', 'SF')]

In [3]:
# Tokenizing 후 pos-tagger 적용
[kkma.pos(morph) for morph in s1.split(' ')]

[[('비정형', 'NNG'), ('데이터', 'NNG'), ('분석', 'NNG')],
 [('수업', 'NNG'), ('에서', 'JKM'), ('는', 'JX')],
 [('정말', 'MAG')],
 [('많', 'VA'), ('은', 'ETD')],
 [('것', 'NNB'), ('을', 'JKO')],
 [('배우', 'VV'), ('ㅂ니다', 'EFN'), ('.', 'SF')]]

In [4]:
# 위 결과물에서 명사계열만 추출
tmp = [kkma.pos(morph) for morph in s1.split(' ')]
tmp = sum(tmp, [])
tmp

[('비정형', 'NNG'),
 ('데이터', 'NNG'),
 ('분석', 'NNG'),
 ('수업', 'NNG'),
 ('에서', 'JKM'),
 ('는', 'JX'),
 ('정말', 'MAG'),
 ('많', 'VA'),
 ('은', 'ETD'),
 ('것', 'NNB'),
 ('을', 'JKO'),
 ('배우', 'VV'),
 ('ㅂ니다', 'EFN'),
 ('.', 'SF')]

In [5]:
tmp = [morph for morph in tmp if morph[1].find('NN') == 0]
tmp

[('비정형', 'NNG'), ('데이터', 'NNG'), ('분석', 'NNG'), ('수업', 'NNG'), ('것', 'NNB')]

### example
본 예제에서는 konlpy에서 제공하는 "한국법률말뭉치"를 이용하여 DocumentTermMatrix를 만들어본다. 이 예제에서는 356줄의 법률문서가 제공되며 이때 각 줄을 문서로보고 DocumentTermMatrix를 만든다. Term은 명사계열(NN?)로만 한정하며 Term-frequency 기준 상위 100개로 DocumentTermMatrix를 구성한다.

In [6]:
from konlpy.corpus import kolaw
fids = kolaw.fileids()
fobj = kolaw.open(fids[0])

In [7]:
text = fobj.readlines()
print(len(text), text[0:6])

356 ['대한민국헌법\n', '\n', '유구한 역사와 전통에 빛나는 우리 대한국민은 3·1운동으로 건립된 대한민국임시정부의 법통과 불의에 항거한 4·19민주이념을 계승하고, 조국의 민주개혁과 평화적 통일의 사명에 입각하여 정의·인도와 동포애로써 민족의 단결을 공고히 하고, 모든 사회적 폐습과 불의를 타파하며, 자율과 조화를 바탕으로 자유민주적 기본질서를 더욱 확고히 하여 정치·경제·사회·문화의 모든 영역에 있어서 각인의 기회를 균등히 하고, 능력을 최고도로 발휘하게 하며, 자유와 권리에 따르는 책임과 의무를 완수하게 하여, 안으로는 국민생활의 균등한 향상을 기하고 밖으로는 항구적인 세계평화와 인류공영에 이바지함으로써 우리들과 우리들의 자손의 안전과 자유와 행복을 영원히 확보할 것을 다짐하면서 1948년 7월 12일에 제정되고 8차에 걸쳐 개정된 헌법을 이제 국회의 의결을 거쳐 국민투표에 의하여 개정한다.\n', '\n', '       제1장 총강\n', '  제1조 ① 대한민국은 민주공화국이다.\n']


In [8]:
# 위의 문서를 살펴본 결과 전처리가 필요함을 알 수 있음
# 위의 문서를 살펴본 결과 줄바꿈도 리스트의 원소로 들어가있음을 볼 수 있음 줄바꿈을 제거해야함
text = list(map(lambda x : re.sub('\\s+',' ', x), text))
text = list(map(lambda x : x.strip(), text))
text = [line for line in text if line != '']
text[0:6]

['대한민국헌법',
 '유구한 역사와 전통에 빛나는 우리 대한국민은 3·1운동으로 건립된 대한민국임시정부의 법통과 불의에 항거한 4·19민주이념을 계승하고, 조국의 민주개혁과 평화적 통일의 사명에 입각하여 정의·인도와 동포애로써 민족의 단결을 공고히 하고, 모든 사회적 폐습과 불의를 타파하며, 자율과 조화를 바탕으로 자유민주적 기본질서를 더욱 확고히 하여 정치·경제·사회·문화의 모든 영역에 있어서 각인의 기회를 균등히 하고, 능력을 최고도로 발휘하게 하며, 자유와 권리에 따르는 책임과 의무를 완수하게 하여, 안으로는 국민생활의 균등한 향상을 기하고 밖으로는 항구적인 세계평화와 인류공영에 이바지함으로써 우리들과 우리들의 자손의 안전과 자유와 행복을 영원히 확보할 것을 다짐하면서 1948년 7월 12일에 제정되고 8차에 걸쳐 개정된 헌법을 이제 국회의 의결을 거쳐 국민투표에 의하여 개정한다.',
 '제1장 총강',
 '제1조 ① 대한민국은 민주공화국이다.',
 '②대한민국의 주권은 국민에게 있고, 모든 권력은 국민으로부터 나온다.',
 '제2조 ① 대한민국의 국민이 되는 요건은 법률로 정한다.']

In [9]:
tagged_text = [kkma.pos(line) for line in text]

In [10]:
tmp_text = []
for line in tagged_text:
    if len(line) >= 2:
        tmp_text.append([morph[0] for morph in line if morph[1].find('NN') >=0 ])
final_text = [' '.join(tmp) for tmp in tmp_text]
final_text[0:6]

['대한민국 헌법',
 '유구 역사 전통 우리 국민 운동 건립 대한민국 임시 정부 법통 불의 항거 민주 이념 계승 조국 민주 개혁 평화적 통일 사명 입각 정의 인도 동포애 민족 단결 사회적 폐습 불의 타파 자율 조화 바탕 자유 민주적 기본 질서 정치 경제 사회 문화 영역 각인 기회 균등 히 능력 최고 도로 발휘 자유 권리 책임 의무 완수 안 국민 생활 균등 향상 밖 항구적 세계 평화 인류 공영 이바지 자손 안전 자유 행복 확보 것 다짐 년 월 일 제정 차 개정 헌법 이제 국회 의결 국민 투표 개정',
 '장 총 강',
 '대한민국 민주 공화국',
 '대한민국 주권 국민 권력 국민',
 '대한민국 국민 요건 법률']

In [11]:
# Term을 설정 (tf기준 상위 100개 단어 추출)
# 한글자단어제거
from collections import Counter
word_list = sum(tmp_text, [])
word_list = [tmp for tmp in word_list if len(tmp) != 1]
word_count = Counter(word_list)
Vocabulary = word_count.most_common(100)
Vocabulary = [tmp[0] for tmp in Vocabulary]
print(Vocabulary[0:6], len(Vocabulary))

['법률', '대통령', '국가', '헌법', '국민', '국회'] 100


In [12]:
# DocumentTermMatrix 생성
from sklearn.feature_extraction.text import CountVectorizer
dtm = CountVectorizer(vocabulary=Vocabulary)

In [13]:
dtm.fit(final_text)
dtm.vocabulary_ # DocumentTermMatrix의 Term
len(dtm.vocabulary_) # Term의 개수

100

In [14]:
my_dtm = dtm.transform(final_text).toarray()

In [15]:
my_dtm.shape

(344, 100)